[![在 Colab 中打开](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-3/dynamic-breakpoints.ipynb) [![在 LangChain Academy 中打开](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/58239526-lesson-4-dynamic-breakpoints)


# 动态断点 

## 回顾

我们讨论了 human-in-the-loop 的动机：

(1) `Approval`（批准）- 我们可以中断智能体，向用户展示状态，并允许用户接受某个动作

(2) `Debugging`（调试）- 我们可以回滚图，以复现或规避问题

(3) `Editing`（编辑）- 你可以修改状态

我们讲解了断点作为在特定步骤停止图的一般方法，从而支持 `Approval` 这样的场景。

我们还展示了如何编辑图状态，并引入人类反馈。

## 目标

断点通常由开发者在编译图时针对特定节点设置。

不过，有时让图**动态地**中断自身会更有帮助！

这是一种内部断点，可以通过 [`NodeInterrupt`](https://langchain-ai.github.io/langgraph/how-tos/human_in_the_loop/dynamic_breakpoints/#run-the-graph-with-dynamic-interrupt) 实现。

这样做有几个好处：

(1) 你可以有条件地触发它（在节点内部基于开发者定义的逻辑）。

(2) 你可以向用户说明为什么被中断（通过把任意信息传递给 `NodeInterrupt`）。

我们来构建一个图，当输入长度超过阈值时抛出 `NodeInterrupt`。


In [1]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain_openai langgraph_sdk

In [ ]:
from IPython.display import Image, display

from typing_extensions import TypedDict
from langgraph.checkpoint.memory import MemorySaver
from langgraph.errors import NodeInterrupt
from langgraph.graph import START, END, StateGraph


class State(TypedDict):
    input: str


def step_1(state: State) -> State:
    print("---Step 1---")
    return state


def step_2(state: State) -> State:
    # Let's optionally raise a NodeInterrupt if the length of the input is longer than 5 characters
    if len(state["input"]) > 5:
        raise NodeInterrupt(
            f"Received input that is longer than 5 characters: {state['input']}"
        )

    print("---Step 2---")
    return state


def step_3(state: State) -> State:
    print("---Step 3---")
    return state


builder = StateGraph(State)
builder.add_node("step_1", step_1)
builder.add_node("step_2", step_2)
builder.add_node("step_3", step_3)
builder.add_edge(START, "step_1")
builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)

# Set up memory
memory = MemorySaver()

# Compile the graph with memory
graph = builder.compile(checkpointer=memory)

# View
# display(Image(graph.get_graph().draw_mermaid_png()))

用一个长度超过 5 个字符的输入来运行这个图吧。


In [4]:
initial_input = {"input": "hello world"}
thread_config = {"configurable": {"thread_id": "1"}}

# Run the graph until the first interruption
for event in graph.stream(initial_input, thread_config, stream_mode="values"):
    print(event)

{'input': 'hello world'}
---Step 1---
{'input': 'hello world'}


/var/folders/zr/zsgg61wn27zbcvccfss34hpw0000gq/T/ipykernel_35915/1678663081.py:18: LangGraphDeprecatedSinceV10: NodeInterrupt is deprecated. Please use `langgraph.types.interrupt` instead. Deprecated in LangGraph V1.0 to be removed in V2.0.
  raise NodeInterrupt(f"Received input that is longer than 5 characters: {state['input']}")


如果我们此时查看图状态，会看到下一步要执行的节点是 `step_2`。


In [5]:
state = graph.get_state(thread_config)
print(state.next)

('step_2',)


可以看到 `Interrupt` 已记录在状态中。


In [6]:
print(state.tasks)

(PregelTask(id='5b7023a6-831a-eac2-a7bd-20a572cd9a86', name='step_2', path=('__pregel_pull', 'step_2'), error=None, interrupts=(Interrupt(value='Received input that is longer than 5 characters: hello world', id='placeholder-id'),), state=None, result=None),)


我们可以尝试从断点恢复图。

但是，这只会再次运行同一个节点！

除非我们修改状态，否则会一直卡住。


In [7]:
for event in graph.stream(None, thread_config, stream_mode="values"):
    print(event)

{'input': 'hello world'}


/var/folders/zr/zsgg61wn27zbcvccfss34hpw0000gq/T/ipykernel_35915/1678663081.py:18: LangGraphDeprecatedSinceV10: NodeInterrupt is deprecated. Please use `langgraph.types.interrupt` instead. Deprecated in LangGraph V1.0 to be removed in V2.0.
  raise NodeInterrupt(f"Received input that is longer than 5 characters: {state['input']}")


In [8]:
state = graph.get_state(thread_config)
print(state.next)

('step_2',)


现在，我们来更新状态。


In [9]:
graph.update_state(
    thread_config,
    {"input": "hi"},
)

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f09aeca-c272-6d76-8002-1718ca883e06'}}

In [10]:
for event in graph.stream(None, thread_config, stream_mode="values"):
    print(event)

{'input': 'hi'}
---Step 2---
{'input': 'hi'}
---Step 3---
{'input': 'hi'}


### 搭配 LangGraph API 的用法

**⚠️ 免责声明**

自从录制这些视频以来，我们已经更新了 Studio，使其可以在本地运行并在浏览器中打开。现在推荐的方式是以这种形式运行 Studio（而不是像视频中展示的桌面应用）。关于本地开发服务器请查看[这里](https://langchain-ai.github.io/langgraph/concepts/langgraph_studio/#local-development-server)的文档，关于本地 Studio 的运行方式请查看[这里](https://langchain-ai.github.io/langgraph/how-tos/local-studio/#run-the-development-server)。在本模块的 `/studio` 目录中，在终端运行以下命令即可启动本地开发服务器：

```
langgraph dev
```

你应该会看到如下输出：
```
- 🚀 API: http://127.0.0.1:2024
- 🎨 Studio UI: https://smith.langchain.com/studio/?baseUrl=http://127.0.0.1:2024
- 📚 API Docs: http://127.0.0.1:2024/docs
```

打开浏览器访问 Studio UI：`https://smith.langchain.com/studio/?baseUrl=http://127.0.0.1:2024`。


In [ ]:
if "google.colab" in str(get_ipython()):
    raise Exception(
        "Unfortunately LangGraph Studio is currently not supported on Google Colab"
    )

我们通过 SDK 与它建立连接。


In [11]:
from langgraph_sdk import get_client

# This is the URL of the local development server
URL = "http://127.0.0.1:2024"
client = get_client(url=URL)

# Search all hosted graphs
assistants = await client.assistants.search()

In [ ]:
thread = await client.threads.create()
input_dict = {"input": "hello world"}

async for chunk in client.runs.stream(
    thread["thread_id"],
    assistant_id="dynamic_breakpoints",
    input=input_dict,
    stream_mode="values",
):

    print(f"Receiving new event of type: {chunk.event}...")
    print(chunk.data)
    print("\n\n")

Receiving new event of type: metadata...
{'run_id': '0199869f-4484-70a3-9d14-d7da6fedd6b5', 'attempt': 1}



Receiving new event of type: values...
{'input': 'hello world'}



Receiving new event of type: values...
{'input': 'hello world'}



Receiving new event of type: values...
{'__interrupt__': [{'value': 'Received input that is longer than 5 characters: hello world', 'id': 'placeholder-id'}]}





In [ ]:
current_state = await client.threads.get_state(thread["thread_id"])

In [ ]:
current_state["next"]

['step_2']

In [ ]:
await client.threads.update_state(thread["thread_id"], {"input": "hi!"})

{'checkpoint': {'thread_id': '77549aef-30f6-4da9-9744-0278173d4480',
  'checkpoint_ns': '',
  'checkpoint_id': '1f09aecc-7fb8-6df8-8002-e3f76c205417'},
 'configurable': {'thread_id': '77549aef-30f6-4da9-9744-0278173d4480',
  'checkpoint_ns': '',
  'checkpoint_id': '1f09aecc-7fb8-6df8-8002-e3f76c205417'},
 'checkpoint_id': '1f09aecc-7fb8-6df8-8002-e3f76c205417'}

In [ ]:
async for chunk in client.runs.stream(
    thread["thread_id"],
    assistant_id="dynamic_breakpoints",
    input=None,
    stream_mode="values",
):

    print(f"Receiving new event of type: {chunk.event}...")
    print(chunk.data)
    print("\n\n")

Receiving new event of type: metadata...
{'run_id': '1ef64c33-fb34-6eaf-8b59-1d85c5b8acc9'}



Receiving new event of type: values...
{'input': 'hi!'}



Receiving new event of type: values...
{'input': 'hi!'}





In [ ]:
current_state = await client.threads.get_state(thread["thread_id"])
current_state

{'values': {'input': 'hi!'},
 'next': ['step_2'],
 'tasks': [{'id': 'e9e592ba-5ae2-5c8c-a540-5d3da121c711',
   'name': 'step_2',
   'path': ['__pregel_pull', 'step_2'],
   'error': None,
   'interrupts': [],
   'checkpoint': None,
   'state': None,
   'result': None}],
 'metadata': {'graph_id': 'dynamic_breakpoints',
  'thread_id': '77549aef-30f6-4da9-9744-0278173d4480',
  'source': 'update',
  'step': 2,
  'parents': {}},
 'created_at': '2025-09-26T15:23:40.659646+00:00',
 'checkpoint': {'checkpoint_id': '1f09aecc-7fb8-6df8-8002-e3f76c205417',
  'thread_id': '77549aef-30f6-4da9-9744-0278173d4480',
  'checkpoint_ns': ''},
 'parent_checkpoint': {'checkpoint_id': '1f09aecc-168c-6da2-8001-6daadc76ac26',
  'thread_id': '77549aef-30f6-4da9-9744-0278173d4480',
  'checkpoint_ns': ''},
 'interrupts': [],
 'checkpoint_id': '1f09aecc-7fb8-6df8-8002-e3f76c205417',
 'parent_checkpoint_id': '1f09aecc-168c-6da2-8001-6daadc76ac26'}